In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from helpers import load_csv_data

In [2]:
yb, data, ids = load_csv_data("data/train.csv")

In [3]:
print(data)

[[ 138.47    51.655   97.827 ...    1.24    -2.475  113.497]
 [ 160.937   68.768  103.235 ... -999.    -999.      46.226]
 [-999.     162.172  125.953 ... -999.    -999.      44.251]
 ...
 [ 105.457   60.526   75.839 ... -999.    -999.      41.992]
 [  94.951   19.362   68.812 ... -999.    -999.       0.   ]
 [-999.      72.756   70.831 ... -999.    -999.       0.   ]]


In [6]:
mean_x = np.mean(data)
data = data - mean_x
std_x = np.std(data)
data = data / std_x

In [7]:
print(data)

[[ 0.72343805  0.52317289  0.62968263 ...  0.40687538  0.3983056
   0.66583024]
 [ 0.77526501  0.56264922  0.64215782 ... -1.90048197 -1.90048197
   0.51064925]
 [-1.90048197  0.77811391  0.69456379 ... -1.90048197 -1.90048197
   0.50609331]
 ...
 [ 0.64728354  0.54363654  0.57896063 ... -1.90048197 -1.90048197
   0.50088224]
 [ 0.62304826  0.44867927  0.56275072 ... -1.90048197 -1.90048197
   0.40401494]
 [-1.90048197  0.57184875  0.56740816 ... -1.90048197 -1.90048197
   0.40401494]]
